# 04 - モデルのマネージドオンラインエンドポイントデプロイ

このノートブックでは、AML モデルレジストリから最新モデルを取得し、マネージドオンラインエンドポイントにデプロイします。

In [ ]:
!pip install azure-core==1.30.2 azure-identity==1.17.1 --force-reinstall
!pip install azure-ai-ml==1.26.3

In [ ]:
# ─── Service Principal の環境変数をセット ──────────────────────────────────
import os
from azure.identity import ClientSecretCredential


# 以下を実際の値に置き換えてください
os.environ['AZURE_SUBSCRIPTION_ID'] = '<your‑subscription‑id>'
os.environ['AZURE_RESOURCE_GROUP']   = '<your‑resource‑group>'
os.environ['AZURE_WORKSPACE_NAME']  = '<your‑workspace‑name>'
os.environ['AZURE_TENANT_ID']       = '<your‑tenant‑id>'
os.environ['AZURE_CLIENT_ID']       = '<your‑client‑id>'
os.environ['AZURE_CLIENT_SECRET']   = '<your‑client‑secret>'
# ─────────────────────────────────────────────────────────────────────────

credential = ClientSecretCredential(
    tenant_id=os.environ['AZURE_TENANT_ID'],
    client_id=os.environ['AZURE_CLIENT_ID'],
    client_secret=os.environ['AZURE_CLIENT_SECRET']
)

In [ ]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Environment as AMLEnvironment
)
import time


# ワークスペースへの接続
ml_client = MLClient(
    credential, 
    os.environ['AZURE_SUBSCRIPTION_ID'],
    os.environ['AZURE_RESOURCE_GROUP'],
    os.environ['AZURE_WORKSPACE_NAME']
)

# モデルの取得
model = ml_client.models.get(name="PointNetFineTuned", version="1")

# ユニークなエンドポイント名を作成
timestamp = int(time.time())
endpoint_name = f"pointnet-seg-endpoint-{timestamp}"

# エンドポイント定義・作成
endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    description="PointNet segmentation model endpoint"
)
ml_client.online_endpoints.begin_create_or_update(endpoint).result()
print(f"🔌 Created endpoint: {endpoint_name}")

# 環境作成
env = AMLEnvironment(
    name="pointnet-custom-env",
    version="1",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
    conda_file="../environments/conda.yml"
)
registered_env = ml_client.environments.create_or_update(env)


# デプロイ定義・作成
deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=model.id,
    environment=registered_env.id,
    code_path="../",
    scoring_script="app/azureml/score.py",
    environment_variables={
        "AZURE_CLIENT_ID":     os.environ["AZURE_CLIENT_ID"],
        "AZURE_CLIENT_SECRET": os.environ["AZURE_CLIENT_SECRET"],
        "AZURE_TENANT_ID":     os.environ["AZURE_TENANT_ID"],
    },
    instance_type="Standard_DS3_v2",
    instance_count=1
)
ml_client.online_deployments.begin_create_or_update(deployment).result()
print("🚀 Deployment 'blue' created")

# エンドポイントのスコアリング URI を表示
endpoint = ml_client.online_endpoints.get(name=endpoint_name)
print(f"🌐 Scoring URI: {endpoint.scoring_uri}")